# ChronoNet

In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn

from load_data import read_data_arrays, data_file_names, standardize_data, data_loader
from models import ChronoNet
from utils import cal_accuracy, evaluate_model 


BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5

print("Reading Data....")
sample_size = 500
data_files = data_file_names(sample_size)
(train_features, val_features, test_features,
 train_labels, val_labels, test_labels, test_lengths) = read_data_arrays(
    data_files)
    
print("Scaling Data....")
train_features, val_features, test_features = standardize_data(
    train_features, val_features, test_features)
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE)
val_iter = data_loader(val_features, val_labels, DEVICE, BATCH_SIZE)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 19
model=ChronoNet(n_chans)
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1), ",Accuracy: ", 
        cal_accuracy(model, train_iter)[0])
    print("Val loss:", val_loss, ",Accuracy: ", 
        cal_accuracy(model, val_iter)[0])

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading Data....
Scaling Data....
Data Loader....
Training Model....
Epoch 1


100%|█████████████████████████████████████████| 692/692 [04:21<00:00,  2.64it/s]


Train loss: 0.6134624477755817 ,Accuracy:  0.7840790960451978
Val loss: 0.5967670480410258 ,Accuracy:  0.7751186440677966
Epoch 2


100%|█████████████████████████████████████████| 692/692 [03:53<00:00,  2.97it/s]


...

Train loss: 0.57588416139859 ,Accuracy:  0.85
Val loss: 0.5925049113504814 ,Accuracy:  0.8061016949152542
Epoch 4


100%|█████████████████████████████████████████| 692/692 [10:09<00:00,  1.13it/s]


Train loss: 0.5709585428582451 ,Accuracy:  0.8510395480225988
Val loss: 0.5953147013982137 ,Accuracy:  0.8055932203389831
Epoch 5


100%|█████████████████████████████████████████| 692/692 [03:29<00:00,  3.30it/s]


Train loss: 0.5691779633994737 ,Accuracy:  0.8547683615819209
Val loss: 0.5908023593745706 ,Accuracy:  0.793593220338983


In [2]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

In [3]:
from collections import Counter

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)

In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_final, yhat_final), confusion_matrix(y_final, yhat_final)

(0.8405797101449275,
 array([[140,  10],
        [ 34,  92]]))

# LSTM

In [5]:
from models import LSTM

learning_rate = 5e-3

model = LSTM(input_size=500, num_channels=19, hidden_units=128)

print("Training Model....")
n_chans = 19
DEVICE = torch.device("cpu")
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
NUM_EPOCHS = 5
BATCH_SIZE = 128

print("started training")

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    print("Train loss:", loss_sum / (t+1), "Accuracy: ", 
        cal_accuracy(model, train_iter)[0])
    print("Val loss:", val_loss, ", Accuracy: ", 
        cal_accuracy(model, test_iter)[0])

Training Model....
started training
Epoch 1


100%|█████████████████████████████████████████| 692/692 [07:19<00:00,  1.57it/s]


Train loss: 0.6562331199990532 Accuracy:  0.7009039548022599
Val loss: 0.649908037629782 , Accuracy:  0.6841378039793663
Epoch 2


100%|█████████████████████████████████████████| 692/692 [08:36<00:00,  1.34it/s]


Train loss: 0.6199005035995748 Accuracy:  0.7613672316384181
Val loss: 0.6462136108501285 , Accuracy:  0.7062146892655368
Epoch 3


...

Train loss: 0.6043506224203661 Accuracy:  0.7858305084745763
Val loss: 0.645477870165133 , Accuracy:  0.7079648734954557
Epoch 4


100%|█████████████████████████████████████████| 692/692 [08:08<00:00,  1.42it/s]


Train loss: 0.5936339165950786 Accuracy:  0.8105762711864407
Val loss: 0.6484245448720222 , Accuracy:  0.7116187668877426
Epoch 5


100%|█████████████████████████████████████████| 692/692 [07:30<00:00,  1.54it/s]


Train loss: 0.5853472212491008 Accuracy:  0.8190621468926553
Val loss: 0.6458646565091376 , Accuracy:  0.7103905674281503


In [6]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
accuracy_score(y_final, yhat_final), confusion_matrix(y_final, yhat_final)

(0.7572463768115942,
 array([[142,   8],
        [ 59,  67]]))